In [1]:
from theano.sandbox import cuda
cuda.use('gpu0')
path = "data/sign_mix_expedition2/"
%matplotlib inline
from __future__ import print_function, division
import utils; reload(utils)
from utils import *
import cv2
import os
import time
from skimage.transform import resize
batch_size=32
vgg = Vgg16()
model=vgg.model
last_conv_idx = [i for i,l in enumerate(model.layers) if type(l) is Convolution2D][-1]
conv_layers = model.layers[:last_conv_idx+1]
def get_bn_da_layers(p):
    return [
        MaxPooling2D(input_shape=conv_layers[-1].output_shape[1:]),
        Flatten(),
        Dropout(p),
        Dense(256, activation='relu'),
        BatchNormalization(),
        Dropout(p),
        Dense(256, activation='relu'),
        BatchNormalization(),
        Dropout(p),
        Dense(24, activation='softmax')
        ]
p=0.8
bn_fc_layers = get_bn_da_layers(p)
hand_sign_model = Sequential(conv_layers + bn_fc_layers)
hand_sign_model.compile(Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
batches = get_batches(path+'train', batch_size=batch_size)
hand_sign_model.load_weights(path+'models/da_conv8_1_at71_5.h5')
final_labels = {}
for x,y in batches.class_indices.items():
    final_labels[y]=x



WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: GeForce GTX 850M (CNMeM is disabled, cuDNN 5110)
WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using Theano backend.


Found 13712 images belonging to 24 classes.


In [3]:
test_batches = get_batches(path+'test', batch_size=batch_size, shuffle=False)
preds = hand_sign_model.predict_generator(test_batches, test_batches.nb_sample)
for i, x in enumerate(preds[0]):
    print(final_labels[i], x)

Found 1 images belonging to 1 classes.
a 0.0341633
b 0.00264158
c 0.1471
d 0.0193105
e 0.0139841
f 0.0251589
g 0.00602279
h 0.00917796
i 0.18866
k 0.0019959
l 0.0267834
m 0.0131079
n 0.012184
o 0.0245394
p 0.0841798
q 0.237123
r 0.00142381
s 0.00536392
t 0.00797522
u 0.00061747
v 0.000610458
w 0.000603885
x 0.0106905
y 0.126582


In [2]:
video_capture = cv2.VideoCapture(0)
fps = 0
start = time.time()

while True:
    # Capture frame-by-frame
    ret, frame = video_capture.read()
    fps += 1

    # Draw rectangle around face
    x = 250
    y = 82
    w = 350
    h = 350
    cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 255, 0), 3)

    # Crop + process captured frame
    hand_ini = frame[85:430, 253:597]
    cv2.imwrite(path+'imgs/imgs/img-'+str(fps)+'.jpg', hand_ini)
    #hand = np.expand_dims(resize(hand_ini, (224, 224), mode='reflect'), axis=0).transpose(0, 3, 1, 2)
    # hand = resize(hand, (224, 224), mode='reflect').transpose(2,0,1)
    # Make prediction
    #my_predict = hand_sign_model.predict(hand,
    #                              batch_size=1,
    #                              verbose=0)
    test_batches = get_batches(path+'imgs', batch_size=1, shuffle=False)
    my_predict = hand_sign_model.predict_generator(test_batches, test_batches.nb_sample)
    # Predict letter
    top_prd = np.argmax(my_predict)
    for i, x in enumerate(my_predict[0]):
        print(final_labels[i], x)
    # Only display predictions with probabilities greater than 0.5
    if np.max(my_predict) >= 0.10:

        prediction_result = final_labels[top_prd]
        preds_list = np.argsort(my_predict)[0]
        pred_2 = final_labels[preds_list[-2]]
        pred_3 = final_labels[preds_list[-3]]

        width = int(video_capture.get(3) + 0.5)
        height = int(video_capture.get(4) + 0.5)

        # Annotate image with most probable prediction
        cv2.putText(frame, text=prediction_result,
                    org=(width // 2 + 130, height // 2 + 75),
                    fontFace=cv2.FONT_HERSHEY_SIMPLEX,
                    fontScale=5, color=(255, 255, 0),
                    thickness=5, lineType=cv2.CV_AA)

    # Display the resulting frame
    cv2.imshow('Video', frame)
    os.remove(path+'imgs/imgs/img-'+str(fps)+'.jpg')

    # Press 'q' to exit live loop
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

# Calculate frames per second
end = time.time()
FPS = fps/(end-start)
print("[INFO] approx. FPS: {:.2f}".format(FPS))

# Release the capture
video_capture.release()
cv2.destroyAllWindows()

Found 1 images belonging to 1 classes.
a 0.006812
b 0.00134805
c 0.670954
d 0.00634067
e 0.0035739
f 0.00192377
g 0.0117314
h 0.0175633
i 0.00551368
k 0.00142805
l 0.106081
m 0.00295827
n 0.00243461
o 0.0245337
p 0.00989297
q 0.110804
r 0.00140157
s 0.000407093
t 0.000727128
u 0.00144266
v 0.00153643
w 0.00022504
x 0.000413005
y 0.00995416
Found 1 images belonging to 1 classes.
a 0.00576014
b 0.00113623
c 0.656017
d 0.00722336
e 0.00513452
f 0.00259119
g 0.018259
h 0.0272713
i 0.00727398
k 0.00188183
l 0.0952657
m 0.0036963
n 0.00286054
o 0.0305692
p 0.0110667
q 0.104591
r 0.00169357
s 0.00056547
t 0.000794279
u 0.00121299
v 0.00148586
w 0.000221356
x 0.000540579
y 0.0128876
Found 1 images belonging to 1 classes.
a 0.00875731
b 0.00118486
c 0.562778
d 0.00508219
e 0.00447442
f 0.00167542
g 0.0178297
h 0.0344577
i 0.00676393
k 0.00163751
l 0.0724292
m 0.00434085
n 0.00350295
o 0.0310116
p 0.0176286
q 0.208772
r 0.00141806
s 0.000680438
t 0.00106559
u 0.00115981
v 0.00102546
w 0.00016353

a 0.00449251
b 0.000730922
c 0.815293
d 0.0053399
e 0.00298854
f 0.000964041
g 0.00678551
h 0.00726763
i 0.00291148
k 0.000575236
l 0.0323788
m 0.00184712
n 0.0014766
o 0.0326472
p 0.00616516
q 0.0700338
r 0.000802957
s 0.000346003
t 0.000522914
u 0.000664411
v 0.000670774
w 9.4196e-05
x 0.000285496
y 0.00471575
Found 1 images belonging to 1 classes.
a 0.00820586
b 0.00100326
c 0.659114
d 0.00619408
e 0.00521031
f 0.00155012
g 0.0133387
h 0.0179214
i 0.0050852
k 0.00109158
l 0.0627468
m 0.00453118
n 0.00336794
o 0.0402848
p 0.0123335
q 0.141749
r 0.00118465
s 0.000651858
t 0.00108754
u 0.000973922
v 0.000986107
w 0.000167179
x 0.000422397
y 0.0107992
Found 1 images belonging to 1 classes.
a 0.00457886
b 0.000737644
c 0.799489
d 0.00248185
e 0.00219017
f 0.00075457
g 0.00546787
h 0.00824028
i 0.00253069
k 0.000420643
l 0.0213362
m 0.00156988
n 0.00111267
o 0.0282528
p 0.00763101
q 0.106668
r 0.000475995
s 0.000315863
t 0.000405137
u 0.000453622
v 0.000447473
w 7.27012e-05
x 0.000182503


a 0.00547801
b 0.00138703
c 0.68042
d 0.00627209
e 0.00360057
f 0.00215093
g 0.00771113
h 0.0147195
i 0.00530744
k 0.00104836
l 0.0754284
m 0.00255138
n 0.00194629
o 0.021952
p 0.0130546
q 0.142049
r 0.0012988
s 0.000386409
t 0.000570278
u 0.00129805
v 0.00127639
w 0.000185572
x 0.000427985
y 0.00947966
Found 1 images belonging to 1 classes.
a 0.00354151
b 0.000656494
c 0.806157
d 0.00417812
e 0.00232468
f 0.0014231
g 0.00586732
h 0.00865384
i 0.00373692
k 0.000503313
l 0.0401905
m 0.00137266
n 0.000962823
o 0.0221621
p 0.0067078
q 0.0807683
r 0.00058918
s 0.000269079
t 0.000334512
u 0.00049092
v 0.000699503
w 0.000113242
x 0.000237481
y 0.00805924
Found 1 images belonging to 1 classes.
a 0.00539769
b 0.000920413
c 0.674308
d 0.00397069
e 0.00294658
f 0.00132045
g 0.00586009
h 0.0120824
i 0.004654
k 0.000571734
l 0.0334975
m 0.00210109
n 0.00140798
o 0.0244377
p 0.0133569
q 0.200742
r 0.000703801
s 0.000392246
t 0.000497217
u 0.000690134
v 0.000698725
w 0.000127228
x 0.000276567
y 0.00

a 0.00551351
b 0.00770877
c 0.100852
d 0.00135115
e 0.00318607
f 0.000677225
g 0.000669068
h 0.000497582
i 0.00264024
k 0.000124307
l 6.12893e-05
m 0.0109079
n 0.00672721
o 0.0263613
p 0.0833802
q 0.741443
r 0.000687444
s 0.0016671
t 0.00321111
u 0.000863555
v 0.000113886
w 0.000129606
x 0.000782614
y 0.000443203
Found 1 images belonging to 1 classes.
a 0.00243298
b 0.00250469
c 0.028158
d 0.00153731
e 0.000696128
f 0.000207291
g 0.000632831
h 0.000392046
i 0.000777848
k 0.000112256
l 0.00011565
m 0.00345952
n 0.0031683
o 0.00695915
p 0.07345
q 0.869948
r 0.000653819
s 0.000281024
t 0.00173193
u 0.00126315
v 0.000230051
w 0.000211216
x 0.000499863
y 0.000577266
Found 1 images belonging to 1 classes.
a 0.00276421
b 0.0927311
c 0.382756
d 0.0134092
e 0.00850902
f 0.00985917
g 0.000698954
h 0.00088075
i 0.00452974
k 0.000689245
l 0.000472768
m 0.00342863
n 0.00240316
o 0.0184957
p 0.0797173
q 0.355242
r 0.00755463
s 0.000696838
t 0.000964746
u 0.00998089
v 0.00140289
w 0.000807053
x 0.001

a 0.0231735
b 0.0335814
c 0.251183
d 0.027755
e 0.0200859
f 0.0121082
g 0.0125143
h 0.0144645
i 0.0321479
k 0.00695801
l 0.016558
m 0.0169672
n 0.0158166
o 0.0328635
p 0.0663066
q 0.362949
r 0.0131093
s 0.0024139
t 0.00691072
u 0.0149849
v 0.00412464
w 0.00169425
x 0.00330772
y 0.00802209
Found 1 images belonging to 1 classes.
a 0.0190024
b 0.0718894
c 0.189379
d 0.0364007
e 0.0489073
f 0.0306639
g 0.0128154
h 0.0185938
i 0.0488333
k 0.0138043
l 0.017591
m 0.0355764
n 0.0299821
o 0.0306784
p 0.0706088
q 0.232534
r 0.0309764
s 0.00406517
t 0.00896108
u 0.0280388
v 0.00649348
w 0.00252888
x 0.00563757
y 0.00603789
Found 1 images belonging to 1 classes.
a 0.0185235
b 0.0415058
c 0.280635
d 0.0416857
e 0.0294509
f 0.0231883
g 0.0145667
h 0.0224557
i 0.05146
k 0.00978891
l 0.0288046
m 0.0201643
n 0.0168178
o 0.0310911
p 0.0460193
q 0.253235
r 0.0205223
s 0.00243689
t 0.00575637
u 0.0205316
v 0.00613409
w 0.00227713
x 0.00375042
y 0.00919846
Found 1 images belonging to 1 classes.
a 0.0150305

...

a 0.0105572
b 0.051115
c 0.0576294
d 0.0212096
e 0.0616639
f 0.00741542
g 0.0052644
h 0.00267696
i 0.00918642
k 0.00365935
l 0.000876995
m 0.158355
n 0.136796
o 0.0439237
p 0.083231
q 0.228375
r 0.0399454
s 0.007786
t 0.0296144
u 0.0271442
v 0.0025513
w 0.00123719
x 0.00886137
y 0.000924418
Found 1 images belonging to 1 classes.
a 0.00531364
b 0.162681
c 0.0979006
d 0.0399537
e 0.038578
f 0.0233792
g 0.00436494
h 0.00349328
i 0.0124559
k 0.00752327
l 0.00184228
m 0.0404872
n 0.0373392
o 0.0235004
p 0.100288
q 0.214579
r 0.0723883
s 0.00360386
t 0.00877151
u 0.0789306
v 0.00886426
w 0.00329101
x 0.00976594
y 0.000705234
Found 1 images belonging to 1 classes.
a 0.000789818
b 0.121988
c 0.0612583
d 0.104944
e 0.0327182
f 0.0271911
g 0.00237917
h 0.00149868
i 0.00653443
k 0.0218173
l 0.00453776
m 0.00490031
n 0.00676122
o 0.00710343
p 0.00485751
q 0.00551288
r 0.291142
s 0.000786011
t 0.00131929
u 0.243041
v 0.037307
w 0.0035583
x 0.00789354
y 0.000160794
Found 1 images belonging to 1 clas

a 0.00540716
b 0.0851042
c 0.0873049
d 0.0963487
e 0.032018
f 0.0123496
g 0.00478877
h 0.00241319
i 0.0311481
k 0.0221301
l 0.00500948
m 0.0281494
n 0.0348979
o 0.0218639
p 0.0377405
q 0.102533
r 0.16253
s 0.00336943
t 0.00933279
u 0.170683
v 0.0250939
w 0.00515387
x 0.0136164
y 0.00101386
Found 1 images belonging to 1 classes.
a 0.0047585
b 0.0895806
c 0.0936465
d 0.0956334
e 0.0530149
f 0.0147387
g 0.00510813
h 0.00209563
i 0.0281992
k 0.0194982
l 0.00302579
m 0.0401304
n 0.0473796
o 0.0277455
p 0.0219101
q 0.0514973
r 0.205268
s 0.00462671
t 0.0112103
u 0.143076
v 0.0193091
w 0.00363265
x 0.0142356
y 0.00067907
Found 1 images belonging to 1 classes.
a 0.00622379
b 0.147492
c 0.130869
d 0.0365101
e 0.0404047
f 0.0117706
g 0.00399307
h 0.00179273
i 0.023987
k 0.011112
l 0.00102311
m 0.0592319
n 0.0525796
o 0.0442795
p 0.0421141
q 0.136068
r 0.0982955
s 0.0057225
t 0.0148262
u 0.103402
v 0.0142357
w 0.00541838
x 0.00800422
y 0.00064392
Found 1 images belonging to 1 classes.
a 0.0066106

a 0.0832008
b 0.0436709
c 0.098773
d 0.0627519
e 0.0216498
f 0.0199251
g 0.0669783
h 0.026357
i 0.0574183
k 0.0204781
l 0.0413668
m 0.0337124
n 0.0528825
o 0.0318036
p 0.0336357
q 0.127714
r 0.0228024
s 0.00555916
t 0.0406418
u 0.0403287
v 0.0181854
w 0.012467
x 0.0112486
y 0.0264484
Found 1 images belonging to 1 classes.
a 0.0328411
b 0.0522263
c 0.101845
d 0.0972365
e 0.023921
f 0.0602416
g 0.0451227
h 0.0151234
i 0.112354
k 0.027122
l 0.0501927
m 0.0226393
n 0.0273421
o 0.0303051
p 0.0178893
q 0.0462607
r 0.0292687
s 0.00385168
t 0.0230098
u 0.0478688
v 0.0480169
w 0.0398972
x 0.0091455
y 0.0362787
Found 1 images belonging to 1 classes.
a 0.0500971
b 0.0834674
c 0.0741663
d 0.062788
e 0.0154679
f 0.0986858
g 0.0295151
h 0.0125147
i 0.143872
k 0.0165541
l 0.0356209
m 0.0215975
n 0.0242237
o 0.0191621
p 0.0184251
q 0.0568233
r 0.0150878
s 0.00325736
t 0.0248891
u 0.0346502
v 0.0327127
w 0.0632567
x 0.00796666
y 0.0551979
Found 1 images belonging to 1 classes.
a 0.107801
b 0.0392616
c 

a 0.00953676
b 0.120977
c 0.189009
d 0.0855061
e 0.0121227
f 0.0467078
g 0.0239402
h 0.00619197
i 0.0760375
k 0.0256982
l 0.0142904
m 0.0102028
n 0.0119973
o 0.0266478
p 0.00831949
q 0.0193247
r 0.0440857
s 0.00226787
t 0.00759006
u 0.112045
v 0.0883138
w 0.0456222
x 0.00874432
y 0.00482073
Found 1 images belonging to 1 classes.
a 0.0226252
b 0.0831876
c 0.115475
d 0.0605826
e 0.0196377
f 0.0885862
g 0.0237303
h 0.00992343
i 0.153224
k 0.0187152
l 0.0224135
m 0.0254823
n 0.0225495
o 0.0274498
p 0.0194823
q 0.0548808
r 0.0221292
s 0.00375577
t 0.0165902
u 0.0428849
v 0.0476753
w 0.0602843
x 0.00739435
y 0.0313408
Found 1 images belonging to 1 classes.
a 0.0316841
b 0.0327542
c 0.198292
d 0.0923508
e 0.0211562
f 0.0432279
g 0.0392612
h 0.012753
i 0.137763
k 0.0159673
l 0.0369949
m 0.022208
n 0.0239918
o 0.0397819
p 0.0187894
q 0.0695366
r 0.0201684
s 0.00348566
t 0.0173395
u 0.0261716
v 0.0273009
w 0.0209743
x 0.00836697
y 0.0396807
Found 1 images belonging to 1 classes.
a 0.065304
b 0.0

a 0.0462885
b 0.004329
c 0.255133
d 0.00219582
e 0.00836623
f 0.00110231
g 0.00957295
h 0.00458095
i 0.00541971
k 0.000208201
l 0.000435537
m 0.0211066
n 0.0134008
o 0.119408
p 0.0497523
q 0.432389
r 0.000401752
s 0.00594458
t 0.0125294
u 0.000425681
v 0.000129336
w 0.000202012
x 0.00107539
y 0.00560276
Found 1 images belonging to 1 classes.
a 0.00453609
b 0.000388778
c 0.0197711
d 0.00022665
e 0.000524544
f 0.000157969
g 0.000616628
h 0.000599218
i 0.00180133
k 2.85872e-05
l 9.47355e-05
m 0.00492295
n 0.00247843
o 0.0108765
p 0.0882643
q 0.859337
r 4.04667e-05
s 0.000731829
t 0.00176662
u 5.23806e-05
v 1.63052e-05
w 3.44345e-05
x 0.000233605
y 0.00249952
Found 1 images belonging to 1 classes.
a 0.0203583
b 0.00222894
c 0.0626736
d 0.000843206
e 0.00258126
f 0.000493717
g 0.00807723
h 0.0167738
i 0.0029189
k 0.000406492
l 0.00199889
m 0.00526731
n 0.00414778
o 0.0131865
p 0.0790105
q 0.771231
r 0.000296674
s 0.00109457
t 0.00227049
u 0.000451711
v 8.88327e-05
w 5.98514e-05
x 0.00042163

In [2]:
import os

In [14]:
test_batches = get_batches(path+'imgs', batch_size=batch_size, shuffle=False)

Found 1 images belonging to 1 classes.


In [13]:
path

'data/sign_mix_expedition2/'